In [1]:
# based on this guide: https://www.tensorflow.org/get_started/eager

from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

#tf.enable_eager_execution() # must be run on a start up program

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

import datetime
import pandas as pd
import numpy as np
import random


Instructions for updating:
Use the retry module or similar alternatives.
TensorFlow version: 1.7.0
Eager execution: False


In [2]:


# load in the data

metadata_file = '/Users/jaspermeijering/Google Drive/a Study/EPA Study Abroad - Carnegie Mellon University/Courses/CMU - 95845 - Applied Analytics The Machine Learning Pipeline/Machine Learning Pipeline Final Project/Data/FalseNews_Code_Data/data/metadata_anon.txt'

# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

    # Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
day = []
hour = []

for cascade,metadata in cascade_id2metadata.items():
    veracity.append(metadata['veracity'])
    virality.append(metadata['virality'])
    depth.append(metadata['depth'])
    breadth.append(metadata['max_breadth'])
    size.append(metadata['size'])
    verified.append(metadata['verified_list'][0])
    nfollowers.append(metadata['num_followers_list'][0])
    nfollowees.append(metadata['num_followees_list'][0])
    engagement.append(metadata['engagement_list'][0])
    category.append(metadata['rumor_category'])
    day.append(metadata['start_date'].day)
    hour.append(metadata['start_date'].hour)

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category, 'day': day, 'hour': hour})

# Inspect
df.head(5)


#def parse_data(line):
 #   example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
  #features = tf.reshape(parsed_line[:-1], shape=(4,))
   # # Last field is the label
  #  label = tf.reshape(parsed_line[-1], shape=())
  #  return features, label



,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,False,NaN
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,False,NaN
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,False,NaN
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,False,NaN
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,False,NaN


In [3]:
df_saved = df


In [33]:
df = df_saved


### Make data numeric

In [34]:
# Make category numeric
df["category"] = df["category"].astype('category')
df["category_num"] = df.category.cat.rename_categories([1,2,3,4,5,6,7])
df["category_num"] = df["category_num"].astype('int')
print("Transformed categories to numeric values, this is how they match",df.category.unique(),df.category_num.unique())

# Make veracity numeric
df["veracity"] = df["veracity"].astype('category')
df["veracity_num"] = df.veracity.cat.rename_categories([1,2,3])
df["veracity_num"] = df["veracity_num"].astype('int')
print("Transformed veracity to numeric values, this is how they match",df.veracity.unique(),df.veracity_num.unique())

# Make verified numeric
df["verified_cat"] = df["verified"].astype('category')
df["verified_cat"] = df["verified_cat"].cat.set_categories(["True","False","None"])
#df["verified_cat"].isnull().sum() # 136 nan / none values for verified
df.loc[df["verified_cat"].isnull(),'verified'] = "None" # -> leave these cascades in. 
df["verified_num"] = df.verified_cat.cat.rename_categories([1,2,3])
df["verified_num"] = df["verified_num"].astype('int')
print("Transformed verified to numeric values, this is how they match",df.verified.unique(),df.verified_num.unique())

#df.dtypes
df.head()


Transformed categories to numeric values, this is how they match [Politics, Viral Photos/Stories/Urban Legends, War/Terrorism/Shootings, Science/Nature/Tech/Food/Health, Entertainment, Business, Natural Disasters]
Categories (7, object): [Politics, Viral Photos/Stories/Urban Legends, War/Terrorism/Shootings, Science/Nature/Tech/Food/Health, Entertainment, Business, Natural Disasters] [4 6 7 5 2 1 3]
Transformed veracity to numeric values, this is how they match [FALSE, TRUE, MIXED]
Categories (3, object): [FALSE, TRUE, MIXED] [1 3 2]
Transformed verified to numeric values, this is how they match ['None'] [3]


,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality,category_num,veracity_num,verified_cat,verified_num
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,None,NaN,4,1,None,3
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,None,NaN,4,1,None,3
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,None,NaN,4,1,None,3
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,None,NaN,4,1,None,3
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,None,NaN,4,1,None,3


### Drop NaN values for virality. 

In [35]:
print(df.virality.isnull().sum(),"of",len(df),"cascades are single tweet cascades. These tweets were not retweeded. \
We are only interessted in rumours that have been spread and want to predict the veracity based on how information deffused \
These sigle tweets will therefor be dropped from the dataset.")

df = df[np.isfinite(df['virality'])]
df.head(10)

84220 of 126301 cascades are single tweet cascades. These tweets were not retweeded. We are only interessted in rumours that have been spread and want to predict the veracity based on how information deffused These sigle tweets will therefor be dropped from the dataset.


,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality,category_num,veracity_num,verified_cat,verified_num
7777,10703,Viral Photos/Stories/Urban Legends,28,11,25.799399,9,186.0,672.0,23228,MIXED,None,4.003857,6,2,None,3
7778,11783,Science/Nature/Tech/Food/Health,1,9,10.811974,6,313.0,380.0,14827,MIXED,None,2.535338,5,2,None,3
7782,6504,Viral Photos/Stories/Urban Legends,27,13,15.395237,21,518.0,504.0,14129,MIXED,None,4.019705,6,2,None,3
7783,5772,Viral Photos/Stories/Urban Legends,22,8,3.140842,4,189.0,228.0,9972,MIXED,None,3.271008,6,2,None,3
7784,6041,Viral Photos/Stories/Urban Legends,21,8,5.160261,23,174.0,110.0,9526,MIXED,None,3.115942,6,2,None,3
7785,6160,Entertainment,10,8,11.310233,0,846.0,842.0,9124,MIXED,None,2.971147,2,2,None,3
7786,2110,Viral Photos/Stories/Urban Legends,28,14,20.554899,6,542.0,716.0,6203,MIXED,None,5.110521,6,2,None,3
7787,3092,Viral Photos/Stories/Urban Legends,13,10,4.256828,4,364.0,179.0,5882,MIXED,None,3.633757,6,2,None,3
7788,4971,Politics,29,3,4.325858,15,417.0,637.0,5075,MIXED,None,2.041730,4,2,None,3
7789,3374,Viral Photos/Stories/Urban Legends,11,5,29.216077,13,910.0,1140.0,4305,MIXED,None,2.584301,6,2,None,3


### Only keep relevant columns

In [36]:
# Only keep relevant columns
column_names_df = ['veracity_num','breadth', 'category_num', 'day', 'depth', 'engangement', 'hour','nfollowees', 'nfollowers', 'size', 'verified_num', 'virality']
column_names_x =['breadth', 'category_num', 'day', 'depth', 'engangement', 'hour','nfollowees', 'nfollowers', 'size', 'verified_num', 'virality']
column_names_y =['veracity_num']

df = df.loc[:,column_names_df]
df.head()

,veracity_num,breadth,category_num,day,depth,engangement,hour,nfollowees,nfollowers,size,verified_num,virality
7777,2,10703,6,28,11,25.799399,9,186.0,672.0,23228,3,4.003857
7778,2,11783,5,1,9,10.811974,6,313.0,380.0,14827,3,2.535338
7782,2,6504,6,27,13,15.395237,21,518.0,504.0,14129,3,4.019705
7783,2,5772,6,22,8,3.140842,4,189.0,228.0,9972,3,3.271008
7784,2,6041,6,21,8,5.160261,23,174.0,110.0,9526,3,3.115942


In [37]:
#set seed
random_array = np.random.choice(["train", "test"], size= len(df), p=[1./2, 1./2])
df["train_test"] = random_array
df.head(10)

,veracity_num,breadth,category_num,day,depth,engangement,hour,nfollowees,nfollowers,size,verified_num,virality,train_test
7777,2,10703,6,28,11,25.799399,9,186.0,672.0,23228,3,4.003857,test
7778,2,11783,5,1,9,10.811974,6,313.0,380.0,14827,3,2.535338,train
7782,2,6504,6,27,13,15.395237,21,518.0,504.0,14129,3,4.019705,train
7783,2,5772,6,22,8,3.140842,4,189.0,228.0,9972,3,3.271008,train
7784,2,6041,6,21,8,5.160261,23,174.0,110.0,9526,3,3.115942,test
7785,2,6160,2,10,8,11.310233,0,846.0,842.0,9124,3,2.971147,train
7786,2,2110,6,28,14,20.554899,6,542.0,716.0,6203,3,5.110521,test
7787,2,3092,6,13,10,4.256828,4,364.0,179.0,5882,3,3.633757,train
7788,2,4971,4,29,3,4.325858,15,417.0,637.0,5075,3,2.041730,train
7789,2,3374,6,11,5,29.216077,13,910.0,1140.0,4305,3,2.584301,test


In [9]:
#df_x = df.iloc[:,1:]
#df_y = df.iloc[:,0]

#df_y.head(10)
#df_x.head(10)


## Running the model

In [75]:
X_train  = df.loc[df["train_test"] == "train",column_names_x ]
Y_train = df.loc[df["train_test"] == "train",column_names_y ]
X_test  = df.loc[df["train_test"] == "test",column_names_x ]
Y_test = df.loc[df["train_test"] == "test",column_names_y ]

Y_test.head(10)

,veracity_num
7777,2
7784,2
7786,2
7789,2
7790,2
7791,2
7795,2
7798,2
7799,2
7800,2


In [52]:
X_train  = X_train.as_matrix()
X_test = X_test.as_matrix()
Y_train = Y_train.as_matrix()
Y_test = Y_test.as_matrix()



In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation


Using TensorFlow backend.


In [73]:
#keras.utils.to_categorical(Y_train, num_classes=None)

In [68]:
model = Sequential([
    Dense(32, input_dim=11),
    Activation('relu'),
    Dense(4),
    Activation('softmax'),
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 32)                384       
_________________________________________________________________
activation_15 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 4)                 132       
_________________________________________________________________
activation_16 (Activation)   (None, 4)                 0         
Total params: 516
Trainable params: 516
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.fit(X_train, keras.utils.to_categorical(Y_train, num_classes=None),
          epochs=20,
          batch_size=128)

Epoch 1/20
21261/21261 [==============================] - 1s 37us/step - loss: nan - acc: 6.5848e-04
Epoch 2/20
21261/21261 [==============================] - 0s 10us/step - loss: nan - acc: 0.0000e+00
Epoch 3/20
21261/21261 [==============================] - 0s 10us/step - loss: nan - acc: 0.0000e+00
Epoch 4/20
21261/21261 [==============================] - 0s 10us/step - loss: nan - acc: 0.0000e+00
Epoch 5/20
21261/21261 [==============================] - 0s 11us/step - loss: nan - acc: 0.0000e+00
Epoch 6/20
21261/21261 [==============================] - 0s 10us/step - loss: nan - acc: 0.0000e+00
Epoch 7/20
21261/21261 [==============================] - 0s 11us/step - loss: nan - acc: 0.0000e+00
Epoch 8/20
21261/21261 [==============================] - 0s 11us/step - loss: nan - acc: 0.0000e+00
Epoch 9/20
21261/21261 [==============================] - 0s 11us/step - loss: nan - acc: 0.0000e+00
Epoch 10/20
21261/21261 [==============================] - 0s 11us/step - loss: nan - acc: 

In [72]:
score = model.evaluate(X_test, keras.utils.to_categorical(Y_test, num_classes=None), batch_size=128)
score

20820/20820 [==============================] - 0s 7us/step


[nan, 0.0]

# For later:

### Stacked LSTM for sequence classification
In this model, we stack 3 LSTM layers on top of each other, making the model capable of learning higher-level temporal representations.

The first two LSTMs return their full output sequences, but the last one only returns the last step in its output sequence, thus dropping the temporal dimension (i.e. converting the input sequence into a single vector).



In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

